In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from math import sqrt
from joblib import Parallel, delayed

In [2]:
df = pd.read_csv('/Users/benkupernk/Documents/GitHub/ML2 Final/ALL_PFAS_CHEMICALS.csv')

In [3]:
df.columns

Index(['Regional Board', 'Public Water System Name', 'Site Name', 'Global ID',
       'Location ID', 'Sample ID', 'Matrix', 'Chemical Name',
       'Chemical Abbreviation', 'Qualifier', 'Value', 'Reporting Limit',
       'Detection Limit', 'Analytical Method Code', 'Lab Notes', 'QRAA',
       'Units', 'Date', 'Treated Drinking Water Sample ID',
       'Treated Drinking Water Qualifier', 'Treated Drinking Water Value',
       'Treated Drinking Water Reporting Limit',
       'Treated Drinking Water Units', 'Treated Drinking Water Date',
       'Field Pt Class', 'Site Use', 'Site Type', 'Facility Type', 'Status',
       'Address', 'City', 'Latitude', 'Longitude'],
      dtype='object')

In [4]:
df.index
df.Units.unique()
df.Matrix.unique()

array(['Liquid', 'Solid', 'Gas'], dtype=object)

In [5]:
def get_val(row):
    val = row['Value']
    if val == 0:
        val = row['Detection Limit']
    return val
df = df[df.Matrix == 'Liquid']
# replace all 0 values with the detection limit
df.Value = df.apply(lambda row: get_val(row), axis = 1)
# get rid of outliers
df = df[df.Value < df.Value.quantile(.95)]
df = df.drop_duplicates(subset = ['Site Name',  'Date', 'Units', 'Sample ID', 'Location ID', 'Chemical Abbreviation'])
df

,Regional Board,Public Water System Name,Site Name,Global ID,Location ID,Sample ID,Matrix,Chemical Name,Chemical Abbreviation,Qualifier,...,Treated Drinking Water Date,Field Pt Class,Site Use,Site Type,Facility Type,Status,Address,City,Latitude,Longitude
0,LAHONTAN RWQCB (REGION 6T),"TRUCKEE-DONNER PUD, MAIN",Tahoe Truckee Sanitation Agency,WDR100034937,EFFLUENT,Final Effluent,Liquid,11-Chloroeicosafluoro-3-oxaundecane-1-sulfonic...,11ClPF3OUDS,ND,...,NaN,ES,Wastewater Treatment Plants,WDR Site,NaN,Active - WDR,13720 Butterfield Drive,Truckee,39.339044,-120.127773
2,LOS ANGELES RWQCB (REGION 4),CAL AMERICAN WATER CO,Hill Canyon WWTP,NPD100051993,HCTP INF,Influent,Liquid,"8:2 Fluorotelomer sulfonic acid (1H, 1H, 2H, 2...",8:2FTS,=,...,NaN,IS,Wastewater Treatment Plants,NPDES,NaN,Active,9600 Santa Rosa Road,Camarillo,34.210560,-118.920000
5,CENTRAL COAST RWQCB (REGION 3),CAL AM WATER COMPANY - MONTEREY,Carmel Reclamation,WDR100029577,CAWDTERT,CAWD-TER500-Q4,Liquid,"8:2 Fluorotelomer sulfonic acid (1H, 1H, 2H, 2...",8:2FTS,<,...,NaN,TPS,WDR Site,WDR Site,NaN,Active - WDR,Highway 1 and Carmel River,Carmel-By-The-Sea,36.539430,-121.919310
6,CENTRAL COAST RWQCB (REGION 3),LOMPOC-CITY WATER UTILITY DIV,Lompoc City Regional Wastewater Reclamation Plant,NPD100051494,GW-003,GW-003 29H3 (1109-20),Liquid,Perfluorobutanesulfonic acid,PFBSA,=,...,NaN,MW,Wastewater Treatment Plants,NPDES,NaN,Active,West 1801 Central Avenue,Lompoc,34.662130,-120.483039
9,CENTRAL VALLEY RWQCB (REGION 5F),"WOODLAKE, CITY OF",Woodlake WWTF,WDR100036550,EFFLUENT,Plant Effluent,Liquid,9-Chlorohexadecafluoro-3-oxanonane-1-sulfonic ...,9ClPF3ONS,ND,...,NaN,ES,Wastewater Treatment Plants,WDR Site,NaN,Active - WDR,811 South Valencia Boulevard,Woodlake,36.401010,-119.098920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886362,NaN,SAN BERNARDINO CITY,EPA WELL 110,W0603610039,CA3610039_066_066,NaN,Liquid,Perfluoro(2-ethoxyethane) sulfonic acid (PFEESA),PFEESA,<,...,10/23/2023,PUBW,Drinking Water Wells,DDW Well,NaN,NaN,NaN,SAN BERNARDINO,34.123804,-117.310700
886363,NaN,CAL-WATER SERVICE CO.-CHICO,WELL 09-03,W0600410002,CA0410002_147_147,NaN,Liquid,"4:2 Fluorotelomer sulfonic acid (1H, 1H, 2H, 2...",4:2FTS,<,...,NaN,PUBW,Drinking Water Wells,DDW Well,NaN,NaN,NaN,CHICO,39.730809,-121.833838
886365,NaN,HOOD WATER MAINTENCE DIST [SWS],WELL W-25,W0603400101,CA3400101_003_003,NaN,Liquid,Perfluorobutanoic acid,PFBA,<,...,NaN,PUBW,Drinking Water Wells,DDW Well,NaN,NaN,NaN,HOOD,38.366202,-121.511872
886366,NaN,"BAKERSFIELD, CITY OF",WELL CBK 01-02 - RAW,W0601510031,CA1510031_043_043,NaN,Liquid,Perfluorooctanoic sulfonate,PFOS,<,...,NaN,PUBW,Drinking Water Wells,DDW Well,NaN,NaN,NaN,BAKERSFIELD,35.339947,-119.075068


In [6]:
# sorted(df['Site Use'].unique())
# source_list = ['Airport',  'Military Cleanup Site', 'Military Privatized Site', 'Military UST Site'
#                'Industrial', 'Industrial - Bulk Fuel Terminal/Refinery', 'Industrial - Chrome Plating',
#               'Land Disposal Site', 'MSW Landfill', 'Other Landfill']
# df = df[df['Site Use'].isin(source_list)]
# sorted(df['Site Use'].unique())

In [7]:
idx_columns = ['Site Name', 'Site Use', 'Date', 'Units', 'Sample ID', 'Location ID']
len_idx = len(idx_columns)
dfp = df.pivot(index = idx_columns, columns='Chemical Abbreviation', values=['Value'])
dfp = dfp.reset_index(level = ['Site Name', 'Site Use', 'Date', 'Units', 'Sample ID', 'Location ID'])
dfp.columns = list(dfp.columns.get_level_values(0)[:len_idx ]) + list(dfp.columns.get_level_values(1)[len_idx :])
#dfp.columns = [col.replace(':', '_') for col in dfp.columns]
dfp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31990 entries, 0 to 31989
Data columns (total 51 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Site Name    30760 non-null  object 
 1   Site Use     31990 non-null  object 
 2   Date         31990 non-null  object 
 3   Units        31990 non-null  object 
 4   Sample ID    8288 non-null   object 
 5   Location ID  31990 non-null  object 
 6   10:2FTS      1126 non-null   float64
 7   11ClPF3OUDS  26838 non-null  float64
 8   3:3FTCA      949 non-null    float64
 9   4:2FTS       14171 non-null  float64
 10  5:3FTCA      793 non-null    float64
 11  6:2FTS       13341 non-null  float64
 12  7:3FTCA      820 non-null    float64
 13  8:2FTS       13904 non-null  float64
 14  9ClPF3ONS    26847 non-null  float64
 15  ADONA        26456 non-null  float64
 16  ETFOSA       4130 non-null   float64
 17  ETFOSE       3927 non-null   float64
 18  HFPA-DA      22767 non-null  float64
 19  HFPO

In [8]:
drop_list =[]
drop_limit = dfp.shape[0] * .5
for col in dfp.columns[len_idx:]:
    count = dfp[col].notna().sum()
    if count < drop_limit:
        drop_list.append(col)
dfp_clean = dfp.drop(columns = drop_list)
dfp_clean = dfp_clean.dropna(subset = dfp_clean.columns[len_idx:], how = 'any')
dfp_clean[dfp_clean.columns[len_idx:]] =  dfp_clean[dfp_clean.columns[len_idx:]].div(dfp_clean[dfp_clean.columns[len_idx:]].sum(axis = 1), axis = 0)
for site in dfp_clean['Site Name'].unique():
    df_temp = dfp_clean[dfp_clean['Site Name'] == site]
    if df_temp.shape[0] >= 25:
        print(site)
        print(df_temp.shape[0])

201-W7
28
ARDEN WELL 18
32
CLEAN HARBORS BUTTONWILLOW,INC
25
Edward C. Little WRP- West Coast Basin Barrier Project - Expansion Phase III
145
LOST CANYON WELL 02A
26
MAIN WELL
59
Oxnard Advanced Water Purification Facility (AWPF)
25
Radius Recycling (Former SCHNITZER STEEL PRODUCTS COMPANY, INC.)
28
SAND CANYON WELL
26
SITE 03 - WELL 02 (NORTH)
33
SITE 4 - EAST WELL
26
SITE 8 - EAST WELL
34
WATSONVILLE CLASS III LANDFILL
58
WELL
43
WELL #1
41
WELL 01
363
WELL 01 - RAW
29
WELL 01B
62
WELL 01D
47
WELL 01E
31
WELL 02
202
WELL 02A
26
WELL 03
121
WELL 03A
29
WELL 04
105
WELL 05
149
WELL 05A
28
WELL 06
124
WELL 07
85
WELL 07-01
28
WELL 08
93
WELL 08-01
26
WELL 09
79
WELL 09-01
33
WELL 1
44
WELL 10
106
WELL 11
72
WELL 11-01
26
WELL 11A
53
WELL 11C
44
WELL 11D
46
WELL 12
73
WELL 13
99
WELL 14
105
WELL 15
83
WELL 16
58
WELL 17
47
WELL 18
55
WELL 19
43
WELL 2
46
WELL 201-W8
26
WELL 201-W9
28
WELL 21
28
WELL 22
39
WELL 24
41
WELL 25
38
WELL 26
32
WELL 30
27
WELL 31-01
44
WELL 59-01
25
WELL 63-01


In [9]:
print(dfp_clean.shape)
corr = dfp_clean[dfp_clean.columns[len_idx:]].corr()
corr.style.background_gradient(cmap='coolwarm')

(14442, 24)


,11ClPF3OUDS,9ClPF3ONS,ADONA,HFPA-DA,NETFOSAA,NMEFOSAA,PFBSA,PFDOA,PFHA,PFHPA,PFHXSA,PFNA,PFNDCA,PFOA,PFOS,PFTEDA,PFTRIDA,PFUNDCA
11ClPF3OUDS,1.000000,0.922164,0.791696,0.515901,0.626171,0.579420,-0.426334,0.919399,-0.523572,0.176720,-0.342643,0.756416,0.845433,-0.695683,-0.624063,0.895856,0.768424,0.790199
9ClPF3ONS,0.922164,1.000000,0.816124,0.502238,0.645361,0.620500,-0.420180,0.921135,-0.532371,0.156268,-0.391529,0.751829,0.873354,-0.672200,-0.620597,0.880600,0.783518,0.797990
ADONA,0.791696,0.816124,1.000000,0.446698,0.550733,0.526700,-0.383905,0.798987,-0.459598,0.122524,-0.346185,0.640555,0.736892,-0.597104,-0.557649,0.768695,0.684440,0.691373
HFPA-DA,0.515901,0.502238,0.446698,1.000000,0.435159,0.437047,-0.323072,0.547807,-0.331207,-0.002269,-0.281520,0.399300,0.466825,-0.476938,-0.468825,0.559685,0.600760,0.593589
NETFOSAA,0.626171,0.645361,0.550733,0.435159,1.000000,0.924836,-0.439898,0.698612,-0.407765,0.059910,-0.386617,0.522296,0.620455,-0.566611,-0.566044,0.717748,0.779360,0.733501
NMEFOSAA,0.579420,0.620500,0.526700,0.437047,0.924836,1.000000,-0.424660,0.685866,-0.395767,0.035321,-0.388691,0.497753,0.605717,-0.546223,-0.553582,0.676599,0.772403,0.728934
PFBSA,-0.426334,-0.420180,-0.383905,-0.323072,-0.439898,-0.424660,1.000000,-0.454552,0.267190,-0.002155,0.151581,-0.383055,-0.433377,0.278585,0.100134,-0.465281,-0.468934,-0.459546
PFDOA,0.919399,0.921135,0.798987,0.547807,0.698612,0.685866,-0.454552,1.000000,-0.542036,0.144860,-0.403430,0.766906,0.901935,-0.696128,-0.657993,0.936398,0.867125,0.881675
PFHA,-0.523572,-0.532371,-0.459598,-0.331207,-0.407765,-0.395767,0.267190,-0.542036,1.000000,0.221412,-0.034077,-0.472790,-0.463960,0.443160,-0.031324,-0.508710,-0.473112,-0.489661
PFHPA,0.176720,0.156268,0.122524,-0.002269,0.059910,0.035321,-0.002155,0.144860,0.221412,1.000000,-0.273431,0.138187,0.165200,0.027373,-0.399872,0.166190,0.028864,0.051647


In [10]:
#df.div(df.sum(axis=1), axis=0)


In [11]:
def gini(vals):
    total = sum(vals)
    vals = [(val / total)**2 for val in vals]
    return 1- sum(vals)


def group_gini(gini_vals, samps_per_cluster):
    """The idea here is to take a list of gini values (one for each cluster) and return a single number
    That represtent the overall gini of all the clusters. I.E a number I can use to compare levles of the 
    dendegram and see which is the best sorted"""
    # By Subtracting .5 and taking the absolute valu larger values now represent a better score
    gini_vals = [val - .5 for val in gini_vals]
    gini_vals = [abs(val) for val in gini_vals]
    # after the above the gini has a range of 0 to .5. Make it 0 to 1 because it makes more sence
    gini_vals = [val *2 for val in gini_vals]

    # now the number of sample in each cluster needs to be taken into acount I.E a perfectly sorted cluster with 2 samples
    # in it should not be weighted equaly with a badly sored cluster with 500 samples in it
    total_samps = sum(samps_per_cluster)
    weighted_gini_vals = [gini_val * (num_samps/total_samps) for gini_val, num_samps in zip(gini_vals, samps_per_cluster)]
    # add them all up and return. Bigger equals better
    return sum(weighted_gini_vals)

In [13]:
data = dfp_clean[dfp_clean.columns[len_idx:]].to_numpy()

def calculate_groups(n):
    groups = AgglomerativeClustering(n_clusters = n).fit_predict(data)
    dfp_clean['groups'] = groups

    gini_per_cluster = []
    samples_per_cluster = []
    for group in set(groups):
        df_temp = dfp_clean[dfp_clean.groups == group]
        site_type_count = {}
        
        for site_type in df_temp['Site Name'].unique():
            df_temp_site = df_temp[df_temp['Site Name'] == site_type]
            site_type_count[site_type] = df_temp_site.shape[0]
            
        gini_value = gini(site_type_count.values())
        gini_per_cluster.append(gini_value)
        samples_per_cluster.append(df_temp.shape[0])
        
    return group_gini(gini_per_cluster, samples_per_cluster)

agg_gini_list = Parallel(n_jobs=3)(delayed(calculate_groups)(n) for n in range(2, 10))
agg_gini_list

[0.9931719691541279,
 0.9905526331237359,
 0.9892021482597722,
 0.9880189447875504,
 0.9872364030920567,
 0.9861797786283073,
 0.9851550690394805,
 0.9824630105674388]

In [ ]:
# data = dfp_clean[dfp_clean.columns[len_idx:]].to_numpy()

# linkage_data = linkage(data, method='ward', metric='euclidean')
# dendrogram(linkage_data)
# plt.show()

In [ ]:
# data = dfp_clean[dfp_clean.columns[len_idx:]].to_numpy()
# clusters_dict = {}
# for n in [2, 3, 4, 5]:
#     groups = AgglomerativeClustering(n_clusters = n).fit_predict(data)
#     dfp_clean['groups'] = groups
#     lables_list = []

#     group_dict = {}
#     gini_per_cluster = []
#     for group in set(groups):
#         df_temp = dfp_clean[dfp_clean.groups == group]
#         site_dict = {}

#         gini_list = []
#         for site_type in df_temp['Site Use'].unique():
#             df_temp_site = df_temp[df_temp['Site Use'] == site_type]
#             site_dict[site_type] = df_temp_site.shape[0]
#             gini_list.append(df_temp_site.shape[0])
#         gini_per_cluster.append(gini(gini_list))
#         print(gini_per_cluster)   
#         group_dict[group] = site_dict
        
#     clusters_dict[n] = (site_dict)
#     print(gini_list)
#     print(gini(gini_list))
#     break
        

In [ ]:
clusters_dict

In [ ]:
clusters_dict[3].values()